In [1]:
import os
import numpy as np 
import zstandard
from tqdm import tqdm
import torch
from scipy import signal
from scipy.ndimage import uniform_filter1d
import matplotlib
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import skimage.filters
import skimage.measure    
from gamutRF.gamutrf_dataset import * 

from gamutrf.sample_reader import get_reader
from gamutrf.utils import parse_filename 

In [2]:
label_dirs= {
     #'drone': ['data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/','data/gamutrf-birdseye-field-days/pdx_field_day_2022_05_26/worker1/gamutrf/'], 
     #'wifi_2_4': ['data/gamutrf-pdx/07_21_2022/wifi_2_4/'], 
#     'wifi_5': ['data/gamutrf-pdx/07_21_2022/wifi_5/']
   # 'mavic3': ['data/office/']
    'home': ['data/home/']
}
sample_secs = 0.02
nfft = 512
batch_size = 8
num_workers = 0


dataset = GamutRFDataset(label_dirs, sample_secs=sample_secs, nfft=nfft, feat="iq")



Generating IQ from 0.02 second chunks of signal data with 512 length FFTs.

Loading filenames...
label='home', 1 files
labeled_filenames={'home': ['data/home/samples_1678913745_950000000Hz_100000000sps.raw.zst']}

Loading data...


  0%|                                                                                       | 0/1 [00:00<?, ?it/s]

[]


IndexError: tuple index out of range

In [ ]:
torch.max(dataset[0][0])

In [ ]:
def debug2(dataset, i):
    idx = dataset.idx[i]
    label_str = idx[0]
    filename = idx[1]
    byte_offset = idx[2]
    req_center, sample_rate, sample_dtype, sample_bytes, sample_type, sample_bits = parse_filename(filename)
    samples = dataset.read_recording(filename, sample_rate, sample_dtype, sample_bytes, dataset.sample_secs, seek_bytes=byte_offset)
    
    f, psd = signal.welch(samples, sample_rate, window=signal.hann(dataset.nfft, sym=False), nperseg=dataset.nfft, return_onesided=False)
    f = np.fft.fftshift(f)
    psd = np.fft.fftshift(psd)
    
    plt.figure(figsize=(12,6))
    
    
    smooth_psd = uniform_filter1d(psd, 5)
    plt.plot(f,smooth_psd)
    plt.plot(f, psd)
    peaks, _ = signal.find_peaks(smooth_psd, distance=40, prominence=0.0001, height=np.mean(psd))
    plt.plot(f[peaks], smooth_psd[peaks], 'x', markersize=12, markeredgewidth=4, label='peaks')
    
    thresh_top    = np.mean(psd) + 1 * np.std(psd)
    plt.plot(f,np.ones_like(f)*thresh_top, '--', label='thresh')
    plt.plot(f,np.ones_like(f)*np.median(psd), '--', label='median')
    plt.plot(f,np.ones_like(f)*np.mean(psd), '--', label='mean')
    plt.legend()
    plt.show()
    
    entropy = skimage.measure.shannon_entropy(psd)
    print(f"{signaltonoise(psd)=}")
    print(entropy)
    return samples

debug2(dataset, 1).shape

In [ ]:
dataset.debug(10)

In [ ]:
def signaltonoise(a, axis=0, ddof=0):
#     a = np.asanyarray(a)
#     m = a.mean(axis)
#     sd = a.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, m/sd)
    a = np.asanyarray(a)
    m = np.mean(a)
    sd = np.std(a)
    return np.where(sd == 0, 0, m/sd)

def snr(signal, noise): 
    return np.mean(signal) - np.mean(noise)

for i in np.random.randint(len(dataset), size=10): 
    img, label, S = dataset[i]
    print(f"{i=}")
    print(f"{np.mean(S)=}")
    
    #print(f"{signaltonoise(S)=}")
    original = S.copy() 
    
    # convert the image to grayscale
    #gray_image = skimage.color.rgb2gray(S)
    # blur the image to denoise
    S = (S-np.min(S))/(np.max(S) - np.min(S))
    blurred_image = skimage.filters.gaussian(S, sigma=1.0)
    t = skimage.filters.threshold_otsu(blurred_image)
    
    
    plt.imshow(S, aspect='auto', origin='lower')
    plt.colorbar()
    plt.show()
    
    # convert the image to grayscale
    #gray_image = skimage.color.rgb2gray(S)
    # blur the image to denoise
    blurred_image = skimage.filters.gaussian(S, sigma=1.0)
    t = skimage.filters.threshold_otsu(blurred_image)  
    
    # create a binary mask with the threshold found by Otsu's method
    binary_mask = blurred_image > t

    plt.imshow(binary_mask, cmap="gray", aspect='auto', origin='lower')
    plt.show()
    
    
    selection = S.copy()
    print(f"{np.mean(original[binary_mask])=}")
    print(f"{np.mean(original[~binary_mask])=}")
    print(f"{snr(original[binary_mask], original[~binary_mask])=}")
    
    selection[~binary_mask] = 0

    fig, ax = plt.subplots()
    plt.imshow(selection, aspect='auto', origin='lower')
    plt.show()
    


In [ ]:
np.mean(S)

In [ ]:
len(dataset)